In [2]:
import intake
import xarray as xr
import numpy as np 
from functools import partial
from workflow.scripts.utils import  regrid_global
import matplotlib.pyplot as plt
from workflow.scripts.plotting_tools import global_map
import statsmodels.api as sm

In [3]:

var_id = 'od550dust'
exp_id = 'piClim-control'
table_id = snakemake.config['table_ids'].get(var_id, snakemake.config['table_id_default'])


In [4]:
esm_cat = intake.open_esm_datastore(snakemake.input.catalog)

In [5]:
col = esm_cat.search(experiment_id = exp_id, variable_id=var_id)
mod_id2xdust = esm_cat.search(experiment_id = 'piClim-2xdust', variable_id=var_id).unique()['source_id']

In [6]:
mod_ids = list(col.unique()['source_id'])
mod_id2xdust = list(mod_id2xdust)

mod_ids = [mod_id for mod_id in mod_ids if mod_id in mod_id2xdust]

In [34]:
def regrid_dataset(ds, grid_params, grid_path):

    method=grid_params.get('method','conservative')
    if grid_path:
        out_grid = xr.open_dataset(grid_path)
        ds = regrid_global(ds, out_grid, method=method)
    elif grid_params.get('dxdy',None):
        dxdy = grid_params['dxdy']
        ds= regrid_global(ds, lon=dxdy[0], lat=dxdy[1], method=method,ignore_degenerate=True)
    else:
        print('No outgrid provided!')
    return ds

def get_dod_treshold(data,surfFrac=0.8):
    data_1d = data.values.ravel()
    edcf = sm.distributions.ECDF(data_1d)
#     return edcf
    filtered = np.where(edcf.y>surfFrac, edcf.x, np.nan)
    filtered = filtered[~np.isnan(filtered)]
    return filtered.min()

In [28]:
dsets = {}
for mod_id in mod_ids:
    if snakemake.config['model_specific_variant'].get(exp_id, None):
        memb_id = snakemake.config['model_specific_variant'][exp_id].get(mod_id, snakemake.config['variant_default'])
    else:
        memb_id = snakemake.config['variant_default']
    temp_col = col.search(source_id=mod_id, member_id=memb_id, table_id=table_id)
    ds = temp_col.to_dataset_dict(progressbar=False)
    ds = ds[list(ds.keys())[0]]
    ds = ds.drop('member_id').squeeze() 
    if snakemake.config.get('regrid_params', None) and snakemake.params.get('regrid', True):
        if ds.data_vars.get('lon_bnds') is not None:
            if 'time' in ds.lon_bnds.dims:
                ds = ds.drop('lon_bnds')
        grid_params=snakemake.config['regrid_params']
        regrid_func = partial(regrid_dataset,grid_params = snakemake.config['regrid_params'],
                                                grid_path = grid_params.get('grid_path',None))
    dsets[mod_id] = regrid_func(ds).groupby("time.month").mean().load()





In [33]:
tresh_od

In [37]:
tresh_od = get_dod_treshold(data_nor, .75)
data_nor.where(data_nor>tresh_od).min()

In [38]:
import cartopy.crs as ccrs
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
global_map(ax)
data_nor = dsets['NorESM2-LM']['od550dust'].mean(dim='month')
tresh_od = get_dod_treshold(data_nor)
dusty = data_nor.where(data_nor>tresh_od)
dusty.plot(ax=ax)

In [39]:
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
data = dsets['UKESM1-0-LL']['od550dust'].mean(dim='month')
global_map(ax)
t = get_dod_treshold(data,.75)
dusty = data.where(data>t)
dusty.plot(ax=ax)

In [41]:
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
data = dsets['MPI-ESM-1-2-HAM']['od550dust'].mean(dim='month')
global_map(ax)
t = get_dod_treshold(data,.75)
dusty = data.where(data>t)
dusty.plot(ax=ax)

In [40]:
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
data = dsets['EC-Earth3-AerChem']['od550dust'].mean(dim='month')
global_map(ax)
t = get_dod_treshold(data,.75)
dusty = data.where(data>t)
dusty.plot(ax=ax)

In [ ]:
dist = sm.distributions.ECDF(values_1d)


In [ ]:
filr = np.where(dist.y > 0.75, dist.x,np.nan)

In [ ]:
filr[~np.isnan(filr)][0]

In [ ]:
plt.plot(dist.x, dist.y)

In [ ]:
c1 = np.random.normal(size=(300,1))

In [ ]:
c2 = np.random.normal(2, 1, size=(300,1))

In [ ]:
c2.shape

In [ ]:
c2.shape

In [ ]:
points.s

In [ ]:
data.sel(lon=-179.125, lat= -89.)

In [ ]:
values.values.ravel()[0]

In [ ]:
points[0]

In [ ]:

values_1d.shape